# Loading the Hyperbard data

In [ ]:
from statics import RAWDATA_PATH, DATA_PATH, GRAPHDATA_PATH, META_PATH
from glob import glob
import pandas as pd

### Metadata

In [ ]:
df = pd.read_csv(f"{META_PATH}/playtypes.csv", comment="#")
df.head()

In [ ]:
play_names = list(df.play_name)
play_names

In [ ]:
df.groupby("play_type").count()